In [4]:
# default_exp data_processing.time_series

In [2]:
# export
import numpy as np
import pandas as pd
import sys
import os
from scipy import stats

/Users/hao.yan/.local/share/virtualenvs/chamomile-Jm9o6Nds/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# export
def cast_date(series, to='month', type='str'):
    if to == 'month':
        fmt = '%Y-%m'
        dt_type = 'datetime64[M]'
    elif to == 'day':
        fmt = '%Y-%m-%d'
        dt_type = 'datetime64[D]'
    if type == 'str':
        return pd.to_datetime(series).dt.strftime(fmt)
    elif type == 'datetime':
        return pd.to_datetime(series).astype(dt_type)



In [6]:
#export

def count_time_value(dataframe, ts_col, group_cols, by='day'):
    df = dataframe.copy()
    df[f'ts_{by}'] = cast_date(df[ts_col], to=by)

    groups = [f'ts_{by}']
    groups.extend(group_cols)

    return df.groupby(groups).size().reset_index(name='counts')

In [5]:
#export
def time_over_time(data_frame: pd.DataFrame = None, ts: str=None, value: str=None
                          , groups: str=None, shift: int=1, drop_last_value: bool=False)->pd.DataFrame:
    """
    
    return: series that contains time over time
    """
    df = data_frame.copy()
    if groups:
        last_value_col = 'last_{g_col}_{v_col}'.format(g_col='_'.join(groups), v_col=value)
        df[last_value_col] = df.groupby(groups)[value].shift(shift)
    else:
        last_value_col = 'last_{v_col}'.format(v_col=value)
        df[last_value_col] = df[value].shift(shift)

    tot_col = '{value}_{u}o{u}_{shift}'.format(value=value, u=ts[0], shift=shift)
    df[tot_col] = df[value] / df[last_value_col]
    
    return df[tot_col]